<a href="https://colab.research.google.com/github/yeclairer/boaz/blob/main/BOAZ_17%EA%B8%B0_%EB%B6%84%EC%84%9D_%EA%B3%BD%EC%9C%A4%EA%B2%BD(6%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library Import

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K 

from tensorflow.keras.datasets import cifar10

In [ ]:
# class 개수 지정
NUM_CLASSES = 10

In [ ]:
# CIFAR-10 dataset load
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

- **Assignment 1: X_train, X_test, y_train, y_test의 shape과 형식을 출력해보세요. (각각 출력해도 됩니다!)**

In [ ]:
## 여기에!
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [ ]:
# 기본적으로 image data는 pixel channel마다 0~255 integer로 구성되는데,
# 신경망은 input이 -1~1 범위에서 가장 잘 동작하기 때문에 데이터를 255로 나눈다.
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [ ]:
# image의 integer label을 one-hot encoding vector로 바꾼다.
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

- **Assignment 2: 변환된 y_train, y_test의 shape을 출력해보세요.**

In [ ]:
## 여기에!
print(y_train.shape)
print(y_test.shape)

(50000, 10)
(10000, 10)


- **Assignment 3: X_train에서, index 54의 이미지에서 (12,13) 위치에 해당하는 픽셀의 초록 채널(1) 값을 출력해보세요. (힌트: X_train[?,?,?,?])**

In [ ]:
## 여기에!
X_train[54,12,13,1]

0.36862746

## Modeling

Keras에서 신경망 구조를 정의하는 방법은 **Sequential 모델**과 **함수형 API** 두 가지가 있습니다.  

Sequential 모델은 일렬로 층을 쌓은 네트워크를 빠르게 만들 때 사용하기 좋습니다. 즉 어떤 분기가 없이 이전 층이 그대로 다음 층으로 연결됩니다.  
하지만 한 층의 출력이 여러 개의 별도의 층으로 전달되거나, 한 층이 여러 층으로부터 입력을 받는 등의 가지가 있는 네트워크를 만들기 위해서는 유연성이 많은 함수형 API를 사용하는 것이 좋습니다.  
Keras에서도 단순한 모델을 만들더라도 Sequential 모델보다 함수형 API를 권장합니다. 장기적으로 보면 모델이 점점 복잡한 구조가 될 수 있기 때문입니다.  
함수형 API는 Deep Neural Network 구조를 설계하는 데에 있어 엄청난 자유를 제공합니다.

따라서 본 assignment에서는 함수형 API를 사용하여 Modeling을 진행하였습니다.

In [ ]:
input_layer = Input(shape=(32,32,3))


x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(input_layer)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Conv2D(filters=128, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Flatten()(x)

x = Dense(256)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Dense(64)(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Dropout(rate=0.3)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)


model = Model(input_layer, output_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0     

## Train

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


- **Assignment 4: loss를 categorical_crossentropy로 설정한 이유는 무엇일까요?**
- 답: 다중분류문제이기 때문입니다.

In [ ]:
BATCH_SIZE = 200
EPOCHS = 30

In [ ]:
model.fit(X_train,
          y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          shuffle=True,
          validation_data=(X_test, y_test))

Epoch 1/30
250/250 [==============================] - 293s 1s/step - loss: 1.8673 - accuracy: 0.3612 - val_loss: 2.8128 - val_accuracy: 0.1363
Epoch 2/30
250/250 [==============================] - 296s 1s/step - loss: 1.4612 - accuracy: 0.4740 - val_loss: 1.8449 - val_accuracy: 0.3571
Epoch 3/30
250/250 [==============================] - 291s 1s/step - loss: 1.3026 - accuracy: 0.5335 - val_loss: 1.3170 - val_accuracy: 0.5388
Epoch 4/30
250/250 [==============================] - 290s 1s/step - loss: 1.1918 - accuracy: 0.5720 - val_loss: 1.4963 - val_accuracy: 0.5130
Epoch 5/30
250/250 [==============================] - 292s 1s/step - loss: 1.1035 - accuracy: 0.6088 - val_loss: 1.1153 - val_accuracy: 0.6029
Epoch 6/30
250/250 [==============================] - 290s 1s/step - loss: 1.0415 - accuracy: 0.6326 - val_loss: 0.9596 - val_accuracy: 0.6590
Epoch 7/30
250/250 [==============================] - 290s 1s/step - loss: 0.9902 - accuracy: 0.6496 - val_loss: 0.9283 - val_accuracy: 0.6748

In [ ]:
model.layers[6].get_weights()

[]

## Evaluate

In [ ]:
model.evaluate(X_test, y_test, batch_size=1000)

10/10 [==============================] - 12s 1s/step - loss: 0.6536 - accuracy: 0.7713


[0.6535791754722595, 0.7713000178337097]

[loss, accuracy] 가 출력된다.  
결과를 시각화해 보자.

In [ ]:
CLASSES = np.array(['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck'])

preds = model.predict(X_test)
preds_single = CLASSES[np.argmax(preds, axis=-1)]
actual_single = CLASSES[np.argmax(y_test, axis=-1)]

In [ ]:
import matplotlib.pyplot as plt

n_to_show = 10
indices = np.random.choice(range(len(X_test)), n_to_show)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.5, wspace=0.5)

for i, idx in enumerate(indices):
    img = X_test[idx]
    ax = fig.add_subplot(1, n_to_show, i+1)
    ax.axis('off')
    ax.text(0.5, -0.35, 'pred = ' + str(preds_single[idx]), fontsize=10, ha='center', transform=ax.transAxes) 
    ax.text(0.5, -0.7, 'act = ' + str(actual_single[idx]), fontsize=10, ha='center', transform=ax.transAxes)
    ax.imshow(img)

**- Assignment 5: model및 training 환경을 자유롭게 변경하여 Accuracy 75% 이상을 달성해 보세요!**

In [ ]:
## 화이팅:)